# Week 5: Recurrent neural networks

Text, speech, weather, sensor output and video are but a few examples of the many types of data that is inherently sequential. So how does one predict the next word in a sentence, future temperatures or missing video frames? Using **recurrent neural networks** (RNNs)!

In [1]:
%matplotlib inline

import numpy as np
import random
import sys
import io
from bs4 import BeautifulSoup
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop

Using TensorFlow backend.


## Exercises

#### Modeling text

Text prediction is a good place to start when learning about RNNs, because most of us humans have a pretty well
optimized inner model for text prediction ourselves. We can, therefore, easily assess the performance of a neural
network in executing this task.

Below is some code that loads the screenplay for Tarantino's 1994 film 'Pulp Fiction'. I recommend reading through the
first 20 lines or so to get a feeling for the language and style used (and enjoy probably the best written screenplay
in the history of film).

In [6]:
import requests as rq

response = rq.get("http://www.dailyscript.com/scripts/pulp_fiction.html")
pulpfiction = BeautifulSoup(response.content, "html.parser").getText()
text = BeautifulSoup(response.content, "html.parser").getText()
print(pulpfiction[:2000])



"PULP FICTION" -- by Quentin Tarantino & Roger Avary


                                      "PULP FICTION"

                                            By

                             Quentin Tarantino & Roger Avary

                

               PULP [pulp] n.

               1. A soft, moist, shapeless mass or matter.

               2. A magazine or book containing lurid subject matter and 
               being characteristically printed on rough, unfinished paper.

               American Heritage Dictionary: New College Edition

               INT. COFFEE SHOP – MORNING

               A normal Denny's, Spires-like coffee shop in Los Angeles. 
               It's about 9:00 in the morning. While the place isn't jammed, 
               there's a healthy number of people drinking coffee, munching 
               on bacon and eating eggs.

               Two of these people are a YOUNG MAN and a YOUNG WOMAN. The 
               Young Man has a slight working-class English acce

> **Ex. 5.1.1:** What is the most used symbol in this screenplay and what accuracy would a model constantly predicting this symbol obtain? In other words, what is the "baseline accuracy"?

In [16]:
freq = {}

for i in text:
    if i in freq:
        freq[i]+=1
    else:
        freq[i]=1

print("Counts of symbols")
print(freq)
max(freq, key=freq.get)

print("Baseline Accuracy is: %f" % (freq[' ']/len(text)))

Counts of symbols
{'\n': 7602, '"': 206, 'P': 310, 'U': 694, 'L': 793, ' ': 164787, 'F': 337, 'I': 1757, 'C': 775, 'T': 1418, 'O': 762, 'N': 1598, '-': 184, 'b': 1629, 'y': 2212, 'Q': 19, 'u': 3530, 'e': 11513, 'n': 7182, 't': 9211, 'i': 6751, 'a': 7713, 'r': 5013, 'o': 8344, '&': 5, 'R': 538, 'g': 2341, 'A': 1081, 'v': 703, '\r': 7596, 'B': 755, '[': 1, 'p': 1568, 'l': 4033, ']': 1, '.': 2806, '1': 23, 's': 6413, 'f': 1913, ',': 1601, 'm': 2322, 'h': 6163, '2': 19, 'z': 67, 'k': 1596, 'c': 2595, 'd': 3507, 'j': 126, 'H': 656, 'D': 435, ':': 103, 'w': 1982, 'E': 1510, 'S': 987, '–': 218, 'M': 885, 'G': 368, "'": 1623, '9': 10, '0': 21, 'W': 556, 'Y': 408, ';': 2, '?': 453, '(': 236, ')': 236, 'q': 64, '!': 178, 'K': 131, 'V': 560, 'J': 480, 'x': 145, '3': 9, '7': 11, '4': 20, '5': 30, 'X': 42, '/': 18, '6': 6, 'Z': 56, '#': 8, '8': 3, '%': 1}
Baseline Accuracy is: 0.541059


I've adapted some code for text generation from [this Keras example](https://keras.io/examples/lstm_text_generation/).
I've inserted some questions in the code (look for `Q:`) for you to answer in the exercise below.

In [7]:
# Q1: What is the purpose of this block? When is `char_indices` used? What about `indices_char`?
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# Q2: What is the purpose of this block? What does the `seqlen` and `step` parameters do?
seqlen = 40
step = seqlen
sentences = []
for i in range(0, len(text) - seqlen - 1, step):
    sentences.append(text[i: i + seqlen + 1])

# Q3: What about this block? What is `x` and what is `y`? Why do they have this dimensionality?
x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    # Q3a: What happens in this loop?
    for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
        x[i, t, char_indices[char_in]] = 1
        y[i, t, char_indices[char_out]] = 1


# Q4: Here we build the model. What does the `return_sequences` argument do? Why the dense layer at the end?
model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model.add(Dense(len(chars), activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy']
)

def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)                     #


def on_epoch_end(epoch, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - seqlen - 1)
    
    # Q5: What does diversity do?
    for diversity in [0.2, 0.5, 1.0]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + seqlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, seqlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.
            
            # What is the dimensionality of `preds`? Why do we input `preds[0, -1]` to the `sample` function?
            preds = model.predict(x_pred, verbose=0)
            next_index = sample(preds[0, -1], diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=50,
          callbacks=[print_callback])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/50
7614/7614 [==============================] - 8s 993us/step - loss: 2.2655 - categorical_crossentropy: 2.2655 - accuracy: 0.5356

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "chicken. But you 
                     "
chicken. But you 
                                                                                                                                                                                                                                                                                                                                                                                                                                     
----- diversity: 0.5
----- Generating with seed: "chicken. But you 
                     "
chicken. But you 
                                                                                         

                                                                                                                                                                                                                                                                                                                                                                                                                     
----- diversity: 1.0
----- Generating with seed: "A. remain firmly fixed on the –

     "
A. remain firmly fixed on the –

                        Butch to to she shot, Maminn Wilf, she?

                                                      f here ho stle he pearf ess from go doll olka?

                      into itte – inchop, to stockin)
                                                 mands oof think of the tanc omxy tho ge

                  Limpall LLICS RATKE- – VINCELD"" OOOM – SAIN IWHTCEN (AT:

        
Epoch 6/50
7614/7614 [==============================] - 7s 901us/step - lo

                                                                                                                                                                                                                                                                                                                                                                                                                                          
----- diversity: 0.5
----- Generating with seed: "s Vincent.

                          "
s Vincent.

                                                                                                                                                                                                                                                                                                   JULES
                                                                                      BUTCH
                                   
----- diversity: 1.0
----- Generating with see

                         dripod's meaning from the outtidion out 
                         in said is going fuckin' assed Sody.

                                               JULIS
                         I'll let's gotta mouthin There!

               The PABORTS OF, aby The tird with the gudvers. The P4OWONY fist bag claigs her 
                         don't fai
Epoch 15/50
7614/7614 [==============================] - 7s 912us/step - loss: 0.7780 - categorical_crossentropy: 0.7780 - accuracy: 0.7824

----- Generating text after Epoch: 14
----- diversity: 0.2
----- Generating with seed: "o 
                         believe Mar"
o 
                         believe Marsellus the car in the bad for the 
                         you need a counter and she cassed is a smart of the 
                         a robbery and the street was a shot.

                                                                                                                                                 

                         panchoor, and the morning. She seem 
                         the seen a baby, I'll gonna be back into the 
                         What does his head: "Yes."

                                                                                                                                                                                                                                            
----- diversity: 1.0
----- Generating with seed: "e girl. The 
                         p"
e girl. The 
                         playbald of "Crunse father, the guns out of her tooush mirror 
                         tell timary.

                                     We ad your cafficen Lousealdly and Jules for a toard of a hund and 
                         Why he chairget.

                                     it obht of years. Butch Dangges her a man fills or 
               breffiat in the life.

               
Epoch 20/50
7614/7614 [==============================] 

...

                         continues to yours?

                                     On't doin'.

               Marilyn hops brought ssilving early room.

               Brett has in.

                                     Winston. I think STINGS kilking are how 
                         rig
Epoch 29/50
7614/7614 [==============================] - 7s 973us/step - loss: 0.6839 - categorical_crossentropy: 0.6839 - accuracy: 0.8081

----- Generating text after Epoch: 28
----- diversity: 0.2
"---- Generating with seed: "     nobody knows why Marsellus tossed 
     nobody knows why Marsellus tossed 
                         the man any problems is the room to the 
                         the shot. That's a shoble and you know 
                         motherfucker you to get into a fast 
                         motherfucker and you know what the boys in 
                         the morning to the shot.

                                                                                
----- 

                         takes a smart of a subwart of the phone.

                                     Vincent stands in the fight of Mia, to be 
                         the shoppres through the hole where 
                         that came in there in the hast of the bold, 
                         this when holding this thing of that what the 
                         behind
----- diversity: 1.0
----- Generating with seed: "                   what is it you wanna "
                   what is it you wanna got your 
                         that what around that case.

                                     SPORTSCAN
                         ...zexperivolyan, navon.
                                             THE WOLF
                         They're eat no moraine 20s looks over 
               and she lEQvion lies wating two.

                                     I don't of colless)
     
Epoch 34/50
7614/7614 [==============================] - 7s 954us/step - loss: 0.6638 - catego

                         the bag in the bathroom door, which the streetting to her heart, 
                         motherfucker. If you don't want to talk 
                         the bedroom of the belt, but way down the 
                         the bag in the box and to to 
                         that car.

                                                                                                
----- diversity: 0.5
----- Generating with seed: "s, they'll kill us, 
                  "
s, they'll kill us, 
                                                                                                                                                 JULES
                              (to The Pot.

                                                                                                                                                                                                                            
----- diversity: 1.0
----- Generating with seed: "s, they'l

                         good take you, My. tectious look or somethin' at if me a foot massage. Vinceld you 
                         stow.

               EXT. PTPELDS in her was that most. Mia low 
               accord to him.

               EXT. BEDACAUD OF the ground to lesp to tmed!


Epoch 43/50
7614/7614 [==============================] - 7s 933us/step - loss: 0.6430 - categorical_crossentropy: 0.6430 - accuracy: 0.8171

----- Generating text after Epoch: 42
----- diversity: 0.2
----- Generating with seed: "ide from how it looks, the car's 
     "
ide from how it looks, the car's 
                         the same was your fuckin' do.

                                     VINCENT
                         You were starting to go of the fuck did a tasty mouth with a porter 
                         the way it to the ready in the fronting at the fuckin' seckin' a table want to the floor.

                                                                                        
---

                         Well, you know what the inta tead. 
                         All strike is in the front diskin' 
                         looks at him, to the freakfish looks over a 
                         of his parkes the shower him.

                                     I meant the shot. Then he was three 
                     
----- diversity: 1.0
----- Generating with seed: "o, and a little baby tomato. 
         "
o, and a little baby tomato. 
                         Hello. They be or nobloy. Butch goneanel sits turndy 
                         off the drongt man that in front of the two med 
               on the LOOP from that giving 
               Vince, you'll killey weatifish the readdy hous the frame to men.

                                     What do your fuckin' nomesong, I have 
                         rume begins offer the w
Epoch 48/50
7614/7614 [==============================] - 7s 949us/step - loss: 0.6345 - categorical_crossentropy: 0.6345 - accuracy

> **Ex. 5.1.2**: Add a callback for Tensorboard, so you can log the training process. Start training the network (takes ~10 minutes on my computer). While it's running move on to the next question.

> **Ex. 5.1.3**: Answer the questions in the code above (look for code comments starting with `Q:`).

> **Ex. 5.1.4**: Did the network finish training? Consider the generated text across epochs.
1. In the early batches (0-10), the generated text looks very bad. Can you explain why the low diversity generated text contains almost only the symbol " " (that is, spaces)?
2. The high diversity generated text is messed up too, but in a different way. Explain how.
3. In later batches (20-30) what do you notice is off about the low diversity generated text?

> **Ex. 5.1.5**: For the network trained over all 50 epochs, generate a longer piece of text
(say 5000 symbols long). Use the sentence `text[1486:1526]` as seed (starts with 'YOUNG MAN' ends with 'No, ')
and set diversity to 0.5.
Describe what features of the screenplay and language in general that the network learned in only 50 epochs.
Also describe what serious mistakes it makes.

> **Ex. 5.1.6**: Do the same as above, but for 40 random letters (e.g. smash away on your keyboard) as seed. What happens? Can you explain why?

> **Challenge** Download [this](https://www.yelp.com/dataset/download) Yelp dataset and train a model that predicts rating given a review text!